In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Scrape the Wikipedia Page
website_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

content = requests.get(website_url).text
soup = BeautifulSoup(content, 'lxml')
codes_html = soup.find('table', {'class':'wikitable sortable'})

table_rows = codes_html.find_all('tr')

df_rows = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text.strip() for i in td]
    if row != []:
        df_rows += [row]

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(df_rows, columns=['Postcode', 'Borough', 'Neighborhood'])

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.    
df = df[df['Borough'] != 'Not assigned']

#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df = df.groupby(by=['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df['Neighborhood'] = df.apply(lambda x: x['Borough'] if x['Neighborhood']=='Not assigned' else x['Neighborhood'], axis=1)

# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe
df.shape

(103, 3)